# AIVLE스쿨 1차 미니프로젝트: 서울시 생활정보 기반 대중교통 수요 분석

<img src = "https://github.com/Jangrae/img/blob/master/moving.png?raw=true" width=800, align="left"/>

- 본 과정에서는 실제 사례와 데이터를 기반으로 문제를 해결하는 전체 과정을 자기 주도형 실습으로 진행해볼 예정입니다.
- 앞선 교육과정을 정리하는 마음과 지금까지 배운 내용을 바탕으로 문제 해결을 해볼게요!
- 미니 프로젝트를 통한 문제 해결 과정 'A에서 Z까지', 지금부터 시작합니다!

---

# [미션 2] 구별 유동인구 분석

### 로컬 수행(Anaconda)
- project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [ ]:
# 기본 경로
path = ''

### 구글 콜랩 수행

- 구글 콜랩을 사용중이면 구글 드라이브를 연결합니다.

In [ ]:
# 구글 드라이브 연결, 패스 지정
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/project/'

### 한글 폰트 표시를 위해 설치합니다.

In [ ]:
# 한글 표시를 위한 라이브러리 설치
!pip install koreanize_matplotlib -q

### 필요한 라이브러리를 불러옵니다.

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

### 기본 데이터: 1.2 seoul_moving_month_202408.csv

- 서울 시 구별 이동 2024년 8월 데이터
- 출처: https://data.seoul.go.kr/dataVisual/seoul/seoulLivingMigration.do

---

# 1. 데이터 불러오기

- 분석 대상 데이터를 불러와 데이터프레임으로 선언합니다.

### [참고] 데이터 프레임을 불러오고 변수로 저장(CSV 기준으로 진행)

- csv: pd.read_csv('파일이름.csv')
- txt: pd.read_csv('파일이름.csv', sep='구분자')
- xlsx: pd.read_excel('파일이름.xlsx')
- pickle: pd.read_pickle('파일이름.pkl')

## [실습1] 데이터 불러오기

- 대상 파일: 1.2 seoul_moving_month_202408.csv
- 불러온 데이터는 seoul_moving 데이터프레임으로 선언하세요.
- 데이터 불러올 때 참고할 사항
    - 구분자(sep)는 ',' 입니다
    - UTF-8 인코더를 사용해 주세요(생략 가능)

In [ ]:
# 데이터 불러오기
seoul_moving = pd.read_csv(path + '1.2 seoul_moving_month_202408.csv', sep=',', encoding='UTF-8')

# 확인
seoul_moving

**[데이터 소개]**
- 도착시간: 0 -> 00:00~00:59를 의미
- 시구군 코드: 자치구 코드
- 성별: F(여성), M(남성)
- 나이: 0 ~ 9세, 10 ~ 79세 (5세 단위), 80세 이상으로 구분
- 이동유형: H(야간상주지), W(주간상주지), E(기타)
- 평균 이동 시간(분): 동일 열 내 이동인구의 월 단위 평균 이동 시간
- 이동인구(합): 추정 합산값으로 소수점 표출, 개인정보 비식별화를 위해 3명 미만의 경우 * 처리

In [ ]:
# 데이터프레임 크기(행, 열) 확인
seoul_moving.shape

---

# 2. 기본 정보 확인 및 전처리

- 분석 대상 테이터를 탐색하고 적절한 전처리를 수행합니다.

## [실습2] 기본 정보 확인

- seoul_moving 데이터프레임 정보를 확인하세요.
- head(), tail(), info(), describe() 메서드 등을 활용해 보세요.

In [ ]:
# head() 메서드로 상위 데이터 확인
seoul_moving.head()

**[0번 열의 의미]**

~~~
"2024년 8월 일요일에 11010(종로구)에서 11010(종로구)으로 이동하여 0시 ~ 0시 59분 사이에 도착했고,
 E(기타)에서 E(기타)로 이동한 0 ~ 9세 여성은 총 28.01명이며 이들의 평균 이동시간은 136분이다"
~~~

In [ ]:
# tail() 메서드로 하위 데이터 확인
seoul_moving.tail()

In [ ]:
# info() 메서드로 열 이름, 데이터 개수, 데이터 형식 등 확인
seoul_moving.info()

In [ ]:
# 표시 형식 변경
pd.set_option('display.float_format', '{:.4f}'.format)

In [ ]:
# describe() 메서드로 기술통계정보 확인
seoul_moving.describe()

In [ ]:
# 표시 형식 초기화
pd.reset_option('display.float_format')

## [실습3] 결측치 처리

- NaN이 아닌 결측치를 처리해 보세요.
- '이동인구(합)' 열 데이터 형식은 현재 object, 즉 str 형입니다.
- 3명 미만 이동인구 합은 개인정보 비식별화를 위해 * 처리 되어 있기 때문입니다.

In [ ]:
# '이동인구(합)' 값별 갯수 확인
# 참고: value_counts() 메서드 사용
seoul_moving['이동인구(합)'].value_counts()

- 그러므로 * 로 저장된 값을 나름의 기준으로 변경하세요.(예: '1,5')

In [ ]:
# '이동인구(합)' 열 * 값을 '1.5'로 변경
# 참고: replace() 메서드 사용
seoul_moving.replace('*', '1.5', inplace=True)

# 확인
seoul_moving['이동인구(합)'].value_counts()

- '이동인구(합)' 열의 데이터 형식을 float 형으로 변경하세요

In [ ]:
# '이동인구(합)' 열 데이터 형식을 float으로 변경
# 참고: astype() 메서드 사용
seoul_moving['이동인구(합)'] = seoul_moving['이동인구(합)'].astype(float)

# 확인
seoul_moving.info()

## [실습4] 범주형 데이터 확인

- '이동유형', '요일', '성별' 열의 범줏값 개수를 확인하세요.


In [ ]:
# '이동유형' 범줏값 개수
# 참고: value_counts() 메서드 사용
seoul_moving['이동유형'].value_counts()

In [ ]:
# '요일' 범줏값 개수
# 참고: value_counts() 메서드 사용
seoul_moving['요일'].value_counts()

- 아래 달력을 보고 2024년 8월 요일 개수를 확인하세요.

<img src = "https://github.com/Jangrae/img/blob/master/202408.png?raw=true" width=200, align="left"/>

In [ ]:
# '성별' 범줏값 개수
# 참고: value_counts() 메서드 사용
seoul_moving['성별'].value_counts()

## [실습5] 서울 지역 데이터 준비

- 우리는 서울시의 데이터만 필요합니다.
    - 시군구 코드:광역시도(2자리)+시군구(3자리) / 서울:11, 부산:26, 대구:27, 인천:28, 광주:29, 대전:30, 울산:31)
- 출발과 도착이 모두 서울인 데이터 ('출발 시군구 코드' < 20000 & '도착 시군구 코드' < 20000)만 추출하세요.
- 추출된 결과를 기존 seoul_moving 데이터프레임으로 다시 선언하세요.

In [ ]:
# 서울지역 데이터 추출
# 참고: df = df.loc[(조건1) & (조건)] 형태 구문 사용
cond1 = seoul_moving['출발 시군구 코드'] < 20000
cond2 = seoul_moving['도착 시군구 코드'] < 20000
seoul_moving = seoul_moving.loc[cond1 & cond2]

# 결과 확인
seoul_moving.head()

- 인덱스를 초기화하세요(단, 기존 인덱스를 버리세요).

In [ ]:
# 인덱스 초기화
seoul_moving.reset_index(drop=True, inplace=True)

# 결과 확인
seoul_moving.head()

**[서울 구별 코드]**

~~~
11010: 종로구    11020: 중구    11030: 용산구   11040: 성동구    11050: 광진구
11060: 동대문구  11070: 중랑구  11080: 성북구   11090: 강북구    11100: 도봉구
11110: 노원구    11120:	은평구  11130: 서대문구 11140: 마포구    11150: 양천구
11160: 강서구    11170: 구로구  11180: 금천구   11190: 영등포구  11200:	동작구
11210: 관악구    11220: 서초구  11230: 강남구   11240: 송파구    11250: 강동구
~~~

- 다음 코드를 필히 실행하여 출발/도착 시구군 코드를 출발/도착 자치구 이름으로 변경하세요.

In [ ]:
# 구 코드를 구 이름으로 변경
seoul_moving['출발 자치구'] = seoul_moving['출발 시군구 코드'].map({
      11010: '종로구',   11020: '중구',   11030: '용산구',   11040: '성동구',   11050: '광진구',
      11060: '동대문구', 11070: '중랑구', 11080: '성북구',   11090: '강북구',   11100: '도봉구',
      11110: '노원구',   11120:	'은평구', 11130: '서대문구', 11140: '마포구',   11150: '양천구',
      11160: '강서구',   11170: '구로구', 11180: '금천구',   11190: '영등포구', 11200: '동작구',
      11210: '관악구',   11220: '서초구', 11230: '강남구',   11240: '송파구',   11250: '강동구'})

seoul_moving['도착 자치구'] = seoul_moving['도착 시군구 코드'].map({
      11010: '종로구',   11020: '중구',   11030: '용산구',   11040: '성동구',   11050: '광진구',
      11060: '동대문구', 11070: '중랑구', 11080: '성북구',   11090: '강북구',   11100: '도봉구',
      11110: '노원구',   11120:	'은평구', 11130: '서대문구', 11140: '마포구',   11150: '양천구',
      11160: '강서구',   11170: '구로구', 11180: '금천구',   11190: '영등포구', 11200: '동작구',
      11210: '관악구',   11220: '서초구', 11230: '강남구',   11240: '송파구',   11250: '강동구'})

# '출발/도착 시구군 코드' 열 제거
seoul_moving.drop(['출발 시군구 코드', '도착 시군구 코드'], axis=1, inplace=True)

# 확인
seoul_moving.head()

## [실습6] 구별 유입/유출 인구 분석

- 구별로 집계하여 다음 내용을 확인하세요.
    - 유출 인구가 제일 많은 구는?
    - 유출 인구가 제일 적은 구는?
    - 유입 인구가 제일 많은 구는?
    - 유입 인구가 제일 적은 구는?

In [ ]:
# 유출이 제일 많은 구와 적은 구는? 강남구, 금천구
# 참고: groupby() 메서드 사용
tmp = seoul_moving.groupby(by=['출발 자치구'], as_index=False)['이동인구(합)'].sum()

print('* 최대유출:', tmp.loc[ tmp['이동인구(합)'] == tmp['이동인구(합)'].max(), '출발 자치구'].values)
print('* 최저유출:', tmp.loc[ tmp['이동인구(합)'] == tmp['이동인구(합)'].min(), '출발 자치구'].values)

In [ ]:
# 유입이 제일 많은 구와 적은 구는? 강남구, 금천구
# 참고: groupby() 메서드 사용
tmp = seoul_moving.groupby(by=['도착 자치구'], as_index=False)['이동인구(합)'].sum()

print('* 최대유입:', tmp.loc[ tmp['이동인구(합)'] == tmp['이동인구(합)'].max(), '도착 자치구'].values)
print('* 최저유입:', tmp.loc[ tmp['이동인구(합)'] == tmp['이동인구(합)'].min(), '도착 자치구'].values)

## [실습7] 구별 이동 시간, 이동 인구 분석

- '도착 자치구' 열을 기준으로 각 구별로 '평균 이동 시간(분)', '이동인구(합)' 평균을 집계하세요.
- 평균 집계 결과를 df_mean 데이터프레임으로 선언하세요.

In [ ]:
# 평균 집계 --> df_mean 데이터프레임으로 선언
# 참고: groupby() 메서드 사용, as_index=False 지정
df_mean = seoul_moving.groupby(by='도착 자치구', as_index=False)[['평균 이동 시간(분)','이동인구(합)']].mean()

# 확인
df_mean.head()

- '도착 자치구' 열을 기준으로 각 구별로 '평균 이동 시간(분)', '이동인구(합)' 합을 집계하세요.
- 합 집계 결과를 df_sum 데이터프레임으로 선언하세요.

In [ ]:
# 합 집계 --> df_sum 데이터프레임으로 선언
# 참고: groupby() 메서드 사용, as_index=False 지정
df_sum = seoul_moving.groupby(by='도착 자치구', as_index=False)[['평균 이동 시간(분)','이동인구(합)']].sum()

# 확인
df_sum.head()

- 다음과 같이 df_sum 데이터프레임의 열 이름을 변경하세요.
    - '평균 이동 시간(분)' --> '총 이동 시간', '이동인구(합)' --> '총 이동인구'

In [ ]:
# 열 이름 변경: '평균 이동 시간(분)' --> '총 이동 시간','이동인구(합)' --> '총 이동인구'
# 참고: rename() 메서드 사용
df_sum.rename(columns={'평균 이동 시간(분)': '총 이동 시간','이동인구(합)':'총 이동인구'}, inplace=True)

# 확인
df_sum.head()

- df_mean, df_sum 두 데이터프레임을 조인(merge) 하여 df_seoul_moving 데이터프레임을 선언하세요.

In [ ]:
# df_mean, df_sum 데이터프레임 조인(merge)
# 참고: '도착 자치구' 열이 조인 기준, how='inner' 지정
df_seoul_moving = pd.merge(df_mean, df_sum, how='inner', on='도착 자치구')

# 확인
df_seoul_moving.head()

- df_seoul_moving 데이터프레임의 '도착 자치구' 열 이름을 '자치구'로 변경하세요.

In [ ]:
# 열 이름 변경
df_seoul_moving.rename(columns={'도착 자치구': '자치구'}, inplace=True)

# 확인
df_seoul_moving.head()

- df_seoul_moving 데이터프레임을 df_seoul_moving.csv 파일로 저장하세요.

In [ ]:
# csv 파일로 저장 (파일명: df_seoul_moving.csv)
# 참고: to_csv() 메서드를 사용, index=False 지정
df_seoul_moving.to_csv(path + 'df_seoul_moving.csv', index=False)

---

# 3. 데이터 분석

- 다양한 분석 과정을 통해 인사이트를 도출하세요.

## [실습8] 시각화 분석

- 다양한 열를 기준으로 그래프를 그려보고 인사이트를 도출해보세요.

In [ ]:
# 연령대별 이동 인구 분포 시각화
tmp = seoul_moving.groupby(by='나이', as_index=False)['이동인구(합)'].sum()

plt.figure(figsize=(12, 6))
plt.bar(x=tmp['나이'].astype(str), height=tmp['이동인구(합)'])
plt.title('연령대별 이동 인구 분포', size=20, pad=15)
plt.xlabel('나이')
plt.ylabel('총 이동 인구')
plt.show()

In [ ]:
# 남녀별 평균 이동 시간 시각화
tmp = seoul_moving.groupby(by='성별', as_index=False)['평균 이동 시간(분)'].mean()

plt.figure(figsize=(12, 6))
plt.bar(x=tmp['성별'], height=tmp['평균 이동 시간(분)'])
plt.title('남녀별 평균 이동 시간', size=20, pad=15)
plt.xlabel('성별')
plt.ylabel('평균 이동 시간(분)')
plt.show()

In [ ]:
# 출발 자치구별 총 이동 인구 시각화
tmp = seoul_moving.groupby(by='출발 자치구', as_index=False)['이동인구(합)'].sum()

plt.figure(figsize=(12, 6))
plt.bar(x=tmp['출발 자치구'].astype(str), height=tmp['이동인구(합)'])
plt.title('출발 자치구별 총 이동 인구', size=20, pad=15)
plt.xlabel('출발 자치구')
plt.ylabel('총 이동 인구')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 위 시각화를 통해 알게된 사실을 정리해 보세요.
# 1. 직장인이 많은 연령대에서 이동 인구가 많음.이는 출퇴근을 위해 많이 이동한다는 것을 시사할 수 있습니다.
# 2. 남성이 더 긴 평균 이동 시간이므로, 이는 남성이 더 먼 거리를 이동하거나, 출퇴근을 위해 더 많은 시간을 소비할 수 있음을 시사함.
#    반대로 이는 여성이 일과 가정 사이의 다양한 장소를 짧게 방문할 수 있음을 의미함.
# 3. 강남구, 송파구 등이 인구 밀도, 교통 편의성, 주요 업무지구 또는 주거지역 여부 등 다양한 요인에 출발 이동인구가 많다.